Преподаватель: Константин Башевой

Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [37]:
import pandas as pd
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

# aggregation
result = ratings.groupby('userId').agg({'timestamp': ['min', 'max'], 'rating': 'count'})

#add life_time
result['life_time']= result['timestamp']['max'] - result['timestamp']['min']

# filter
result_rating_count_more_100 = result[result['rating']['count'] > 100]

# mean calculation
result_rating_count_more_100.life_time.mean()

40080507.4496124

Задание 2
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:

rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов

In [40]:


rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [46]:
result_transportation = rzd.merge(auto, how='outer', on='client_id')
result_transportation = result_transportation.merge(air, how='outer', on='client_id') 
result_transportation_with_adress = result_transportation.merge(client_base, how='outer', on='client_id')
result_transportation
# visits_grouped.join(purchases_pivot, how='outer')

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [47]:
result_transportation_with_adress

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

Как бы вы добавили эти сведения в таблицу визитов и покупок?
- Добавил бы в таблицу Визитов данные по времени начала визита и продолжительности визита (период визита).
- Потом бы Добавил Координаты, в визиты, ко которым их удалось снять
- Потом добавил бы в таблицу Покупок+Координат время завершения транзакции
То есть структуру вижу такую:
 id 1
   - vizit 1 ->
   - vizit 2 -> coordinates 2
   - vizit 3 -> coordinates 3 ->purchases 1
   
result_skvoznoy = visits.merge(coordinates, how='left', on=['client_id', 'time_period'])
result_skvoznoy = result_skvoznoy.merge(purchases, how='left', on=['client_id', 'time_period'])

У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Ответ: Ввел бы параметр продолжительности визита(временной отрезок). Если время снятия координат попадает в этот период визита, то эти координаты привязывал бы к этому визиту

Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
Ответ:
Разбил бы время снятия координат на 3 участка:
 - в пути дом-работа-дом (будни с 7-00 до 9-00 + с 17-00..19-00)
 - на работе (плавающее определение для каждого user_id)
 - прочее (выходные)

Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.
Ответ: По этим показателям бы отображал, что показывать в жизненой ситуации User'a, чтобы максимизировать вероятность покупки